# Tidy Data

---

> Structuring datasets to facilitate analysis [(Wickham 2014)](http://www.jstatsoft.org/v59/i10/paper)

Tidy Data (and the closely related topic of reshaping data) is a way of structuring your data and APIs in a consistent (sensible) way to make the data analysis process flow more smoothly.

## The Rules

In a tidy dataset...

1. Each variable forms a column
2. Each observation forms a row

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="ticks", context="talk")
plt.rcParams['figure.figsize'] = 12, 8
pd.options.display.max_rows = 10
%matplotlib inline

## NBA Games

Earlier, I fetched some data

```python
tables = pd.read_html(
  "http://www.basketball-reference.com/leagues/"
  "NBA_2016_games.html"
)
games = tables[0]
games.to_csv('data/games.csv', index=False)
```

In [ ]:
!head -n  5 data/games.csv

So the data is roughly like

| Date        | Visitor Team | Visitor Points | Home Team | Home Points |
| ----------- | ------------ | -------------- | --------- | ----------- |
| 2015-10-07  | Detroit      | 106            | Atlanta   | 94          |
| ...         | ...          | ...            | ...       | ...         |

Plus some extra junk we don't really care about.

## [The Question](http://stackoverflow.com/questions/22695680/python-pandas-timedelta-specific-rows):

> **How many days of rest did each team get between each game?**

Whether or not your dataset is tidy depends on your question. Given our question, Is our data tidy? What is an observation?


<a href="#answer" class="btn btn-default" data-toggle="collapse">Show Answer</a>
<div id="answer" class="collapse">
An observation is a (team, game) pair. So no, we don't have a tidy dataset.
A tidy dataset would be like

<table>
<thead>
<tr class="header">
<th>Date</th>
<th>Team</th>
<th>Home / Away</th>
<th>Points</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>2015-10-07</td>
<td>Detroit</td>
<td>Away</td>
<td>106</td>
</tr>
<tr class="even">
<td>2015-10-07</td>
<td>Atlanta</td>
<td>Home</td>
<td>94</td>
</tr>
<tr class="odd">
<td>...</td>
<td>...</td>
<td>...</td>
<td>...</td>
</tr>
</tbody>
</table>

</div>

Let's do a bit of general cleaning to remove that extra junk before reshaping the data to be tidy.

In [ ]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = (
    pd.read_csv("data/games.csv")
      .rename(columns=column_names)
      .dropna(thresh=4)
      [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
      .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
      .set_index('date', append=True)
      .rename_axis(["game_id", "date"])
      .sort_index()
)
games.head()

Above, we saw that we need to collapse the away / home teams down to two columns: one identifier and one for the value.
Likewise with the points.
We'll also need to repeat the metadata fields, like the date and `game_id`, so that each observation is matched with the correct date.
`pd.melt` does all this for us.

## Melt

![melt](figures/melt.png)

- Collect a variable spread across multiple columns into one, but
- Repeat the metadata to stay with each observation

In [ ]:
tidy = pd.melt(games.reset_index(),
               id_vars=['game_id', 'date'], value_vars=['away_team', 'home_team'],
               value_name='team', var_name='home_away').sort_values(['game_id', 'date'])

tidy.head()

Now the translation from question to operation is direct:

In [ ]:
# How many days of rest for each team?
# For each team...  get number of days between games
tidy.groupby('team').date.diff().dt.days - 1

We haven't seen `.diff` yet. It takes the value in the current row minus the value in the previous. Since `.date` is a `datetime`, the result is a `timedelta`. Timedeltas have that special `.dt` namespace for accessing properties of each timedelta, like the number of days.

In [ ]:
# Assign tidy['rest']
tidy['rest'] = tidy.groupby('team').date.diff().dt.days - 1
tidy.dropna().head()

## `pivot_table`

You can "invert" a `melt` with `pd.pivot_table`

In [ ]:
by_game = (pd.pivot_table(tidy, values='rest',
                          index=['game_id', 'date'],
                          columns='home_away')
             .rename(columns={'away_team': 'away_rest',
                              'home_team': 'home_rest'})
             .rename_axis(None, axis='columns'))
by_game.dropna().head()

`concat` will merge the two dataframes (`games, by_game`), expanding an `axis`, while aligning on the other axis.

In [ ]:
df = pd.concat([games, by_game], axis='columns')
df.dropna().head()

## Two datasets

- `tidy`: For team-level questions
- `df`: For game-level questions

Even though I called one of the variables `tidy`, both datasets are tidy in their own way, it just depends on the question your trying to answer.

For example, let's say we wanted to look at the rest durations, split by home/away, faceted by team. We'd use `tidy` for that since it's a team-level question. We'll facet by `team` (wrapping the columns so it doesn't get too wide. And `map` a barplot with `x=variable` and `y=rest`.

In [ ]:
g = sns.FacetGrid(tidy, col="team", col_wrap=5, hue="team")
g.map(sns.barplot, "home_away", "rest", order=['away_team', 'home_team']);

If we wanted to look at the distribution of the *differences* in rest, we'd use `df` since the difference is a game-level statistic.

In [ ]:
delta = (df['home_rest'] - df['away_rest']).dropna().astype(int)
(delta.value_counts()
    .reindex(np.arange(delta.min(), delta.max() + 1), fill_value=0)
    .sort_index().plot(kind='bar', color='k', width=.9, rot=0, figsize=(12, 6)))
sns.despine()
plt.xlabel("Difference in Rest (home - away)")
plt.grid(axis='y');

<div class="alert alert-success" data-title="Win Percent">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Win Percent</h1>
</div>

<p>Find the win-percent for each team, by whether they're home or away.</p>

Hints:

- You'll need a new variable in `df` called `winning_team` (use `np.where`?). That should have the name of the winning team
- You'll need to `melt` that along with the home / away columns (which column is `id_vars`? Which columns are `value_vars`?)
- A tidy dataset looks like

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>winning_team</th>
      <th>home_or_away</th>
      <th>team</th>
      <th>won</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Detroit Pistons</td>
      <td>away_team</td>
      <td>Detroit Pistons</td>
      <td>True</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Chicago Bulls</td>
      <td>away_team</td>
      <td>Cleveland Cavaliers</td>
      <td>False</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Golden State Warriors</td>
      <td>away_team</td>
      <td>New Orleans Pelicans</td>
      <td>False</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Boston Celtics</td>
      <td>away_team</td>
      <td>Philadelphia 76ers</td>
      <td>False</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Chicago Bulls</td>
      <td>away_team</td>
      <td>Chicago Bulls</td>
      <td>True</td>
    </tr>
  </tbody>
</table>

- With the tidy dataset we can `groupby` and take the mean. Teams play the same number of home and away games, so a simple (un-weighted) average will work.

In [ ]:
%load solutions/tidy_00.py

## Stack / Unstack

- stack: `DataFrame` -> `Series` with `MultiIndex`
- unstack: `Series` with `MultiIndex` -> `DataFrame`

The exact shape of a tidy dataset depends on the question being asked.
Additionally, not all APIs expect tidy data, so you need to convert between "wide" and "long" form data.

Let's create a `rest` varible that's the average `rest` by `home_away` and `date` across teams:

In [ ]:
rest = tidy.groupby(['home_away', 'date']).rest.mean()
rest.head()

`rest` is in "long" form. `DataFrame.plot` for example, expects wide form data, so use `.unstack`

In [ ]:
rest.unstack('home_away').head()

And we can plot the rolling mean:

In [ ]:
rest.unstack('home_away').rolling(7).mean().plot(figsize=(10, 5));

Stack and unstack are inverses:

In [ ]:
rest.unstack("home_away").stack().head()

<div class="alert alert-success" data-title="Home Court Advantage?">
  <h1><i class="fa fa-tasks" aria-hidden="true"></i> Exercise: Home Court Advantage?</h1>
</div>
<p>How much of home court advantage can be explained by rest?</p>

Reproducing [this](http://www.nessis.org/nessis07/Dylan_Small.pdf).
Basically, we want to run a regression of

    point_spread ~ rest_spread + other_stuff

We'll talk about the `other_stuff` later when we get to it.
For now, we need to augment `df` with a couple additional varibles.

If your DataFrame is out of sync, you might want to load in a fresh one, by copying this code into a new cell and running it.

```python
df = pd.read_csv('data/tidy_checkpoint.csv', parse_dates=['date'],
                 index_col=['game_id', 'date'])
```

## Step 0: Outcome variables

Modify `df` to include a couple potential targets

- `home_win`: binary indicator for whether the home team won
- `point_spread`: the home score minus the away score

In [ ]:
%load solutions/tidy_01.py


## Step 1: Team Strength

Most examples I've seen use a "team strength" variable in their regression estimating the home court advantage. We'll grab one from ESPN.

Use RPI from prior year: http://www.espn.com/nba/stats/rpi/_/year/2015 as a measure of team strength

```python
rpi = pd.read_html("http://www.espn.com/nba/stats/rpi/_/year/2015")[0]
```

In [ ]:
raw_rpi = pd.read_csv("data/rpi.csv")
raw_rpi.head()

In [ ]:
rpi_cols = raw_rpi.loc[1]
rpi = (
    raw_rpi.drop([0, 1])
       .rename(columns={str(i): v for i, v in enumerate(rpi_cols)})
       .set_index("TEAM")
       .assign(RPI=lambda x: pd.to_numeric(x.RPI))
)
rpi.head()

Merge `df` and `rpi` on team. One uses just the city, while the other uses the city and team name.
Fortunately, both versions have the same sort order, so we're able to unambigusly map `<city>` to `<city> <team>`.

In [ ]:
mapping = dict(zip(
    sorted(rpi.index),
    sorted(set(df.away_team.unique()))
))
mapping

Create two new columns in `df`

- `home_strength`
- `away_strength`

Using either `.map` or `pd.merge` and the `mapping` varaible we just created.
The strength columns should contain the `RPI` for the home / away team from `rpi`.

In [ ]:
%load solutions/tidy_02.py

## Step 2: Rest Difference

Create a new column `rest_spread` that contains the difference in rest (home - away)

In [ ]:
%load solutions/tidy_03.py


Remove rows with missing values (rest), fixup a dtype for statsmodels

In [ ]:
df = df.dropna()
df.loc[:, 'home_win'] = df['home_win'].astype(int)
df.head()

## Step 3: Sanity Check

Let's do some checks to see if we're on the right track.
Does the home team typically have more rest?

In [ ]:
%load solutions/tidy_drest.py


What's the relationship between the difference in rest and home win percent? I did a barplot, but you can explore it however you want.

In [ ]:
%load solutions/tidy_04.py


We can plot each team's win percent by home or away:

In [ ]:
g = sns.FacetGrid(win_pct.reset_index(), hue='team', size=8, aspect=.5, palette=['k'])
g.map(sns.pointplot, 'home_or_away', 'won',
      order=['away_team', 'home_team']).set(ylim=(0, 1));

## Step 4: Regression

Now we can fit the model using statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
df.head()

In [ ]:
mod = smf.ols('point_spread ~ rest_spread + home_strength + away_strength', data=df)
res = mod.fit()
res.summary()

So an extra day of rest is worth between -.28 and 1.06 points. Not that impresssive.

## Road Trips

Do teams on longer road trips perform less well?
Basically, what is the probability of winning as a function of number of *consecutive* games on the road?

We'll need to compute an `away_streak` variable that contains the number of consecutive away games.
This will reset to 0 each time the team returns home.
For example:

home_away  | away_streak
---------- | -----------
H          | 0
A          | 1
A          | 2
H          | 0
A          | 1
A          | 2
A          | 3

This will need to be done in the context of a `groupby`.
When developing complicated groupby functions, I like to grab and individual group and prototype on that:

In [ ]:
gr = tidy.groupby('team').home_away
v = gr.get_group("Chicago Bulls")  # Use `.get_group` to extract a single group
v.head()

This is an example of a computation that's difficult (though not impossible) to vectorize.
The relative simplicity of the iterative solution (like the skeleton below) outweighs the performance cost.

Fill in the ellipses (`...`) below.

In [ ]:
import unittest
import pandas.util.testing as tm


def compute_away_streaks(v):
    # streaks will contain one entry per game.
    streaks = []
    current_streak = 0

    for row in v:
        if row == 'away_team':
            ...  # what happens here?
        else:
            current_streak = ...  # what should streak be here?
        streaks.append(streak)  # add to the container

    streaks = pd.Series(streaks, index=v.index)
    return streaks

class TestAwayStreak(unittest.TestCase):
    maxDiff = None

    def test_compute_streaks(self):
        result = compute_away_streaks(v).head(10)  # easier to check
        expected = pd.Series([0, 1, 2, 0, 1, 0, 0, 1, 0, 0],
                             index=[1231, 4, 24, 1269, 52, 1299, 1315, 103, 1357, 1381])

        tm.assert_series_equal(result, expected)

suite = unittest.TestLoader().loadTestsFromModule(TestAwayStreak())
unittest.TextTestRunner().run(suite)

In [ ]:
%load solutions/tidy_05.py

Now let's put `compute_away_streaks` to use. Before looking at the next cell, can you tell which `groupby` method we want to use? Recall that it can be

- `.agg`: one output row per group
- `.transform`: one output row per input row (same shape)
- `.apply`: arbitrary output shape

In this case it will be a `.transform`

In [ ]:
streaks = tidy.groupby("team").home_away.transform(compute_away_streaks).astype(int)
tidy['away_streak'] = streaks
tidy.head()

We need that `.astype` since currently `.transform` always returns the same dtype as the input (`object` in this case). There's an open pull request to fix that.

In [ ]:
streaks.value_counts().plot.bar(color='k', width=.9, rot=0)
sns.despine();

And let's go from `tidy` to our game-level data with another `pivot_table`

In [ ]:
game_away_streak = (
    pd.pivot_table(
        tidy, values='away_streak', index=['game_id', 'date'],
        columns='home_away'
    ).rename(columns={'away_team': 'away_streak'}))['away_streak']
df['game_away_streak'] = game_away_streak

A quick sanity check, and it doesn't seem like there's much to the hypothesis:

In [ ]:
sns.barplot('game_away_streak', 'home_win', data=df);

If teams actually performed worse the longer they were on the road, we'd expect gradually declining bars.
Still, we can run the regression:

In [ ]:
m = smf.ols("point_spread ~ rest_spread + game_away_streak + home_strength + away_strength", df)
res = m.fit()
res.summary()

Again, there doesn't seem to be much to the "away streak" hypothesis. The upper bound of the 95% CI is less than 1 point for each additional consecutive game on the road.

## Recap

- Tidy data:
    - one variable per column
    - one row per observation
- Methods:
    - melt / stack: wide to long
    - pivot_table / unstack: long to wide